In [18]:
import json
import csv
import geopy.distance

CellularTags = ["lte","lte-mts","lte-tdd","lte-vodafone","3g","4g","5g","fallback-to-4g","verizon5g"]
SatelliteTags = ["starlink"]
WifiHomeTags = ["system-wifi","wlan","wi-fi","wireless","fixed-wireless","fixed-wireless-2"]
WiredHomeTags = ["cable","ethernet","wiredethernet","fiber","dsl","ftth","ftth-2","ftth-3","200100-ftth-german-telekom",
"adsl","vdsl","adsl-2","vdsl2","adsl2","adsl2-2","free-adsl"]
probeFileName = "/Users/ecenursen/Desktop/CMB-assignment3/20230210.json"

In [19]:
# function to check whether distance between two probes is smaller than 51 km
def isCloseEnough(p1,p2):
    cord1 = (p1["latitude"],p1["longitude"])
    cord2 = (p2["latitude"],p2["longitude"])
    if geopy.distance.geodesic(cord1, cord2).km < 51:
        return True
    else:
        return False

In [20]:
#function to find all connected cellular probes according to tags given
def findCellularProbes(pFile):
    CellularProbes = {"connected":[]}
    for probe in pFile["objects"]:
        if probe["status_name"] == "Connected":
            inserted = False
            for tag in probe["tags"]:
                for ctag in CellularTags:
                    if tag == ctag:
                        CellularProbes["connected"].append(probe)
                        inserted = True
                        break
                if inserted:
                    break

    with open("./cellularProbes.json", "w") as conProbeFile:
        json.dump(CellularProbes, conProbeFile, indent=4,sort_keys=True)

    print(len(CellularProbes["connected"]))

In [21]:
#function to find all connected satellite probes according to tags given
def findSatelliteProbes(pFile):
    SatelliteProbes = {"connected":[]}
    for probe in pFile["objects"]:
        if probe["status_name"] == "Connected":
            if probe["asn_v4"] == 14593 or probe["asn_v6"] == 14593:
                SatelliteProbes["connected"].append(probe)
                continue
            inserted = False
            for tag in probe["tags"]:
                for ctag in SatelliteTags:
                    if tag == ctag:
                        SatelliteProbes["connected"].append(probe)
                        inserted = True
                        break
                if inserted:
                    break

    with open("./satelliteProbes.json", "w") as satProbeFile:
        json.dump(SatelliteProbes, satProbeFile, indent=4,sort_keys=True)

    print(len(SatelliteProbes["connected"]))

In [22]:
#function to find all connected home Wifi probes according to tags given
def findWifiHomeProbes(pFile):
    WifiHomeTags = ["system-wifi","wlan","wi-fi","public-wifi","free-wifi","wireless","adsl","vdsl","adsl-2","vdsl2","adsl2","adsl2-2",
    "wireless-isp","wireless-link","att-wireless-broadband", "wifi-mesh","fixed-wireless","fixed-wireless-2","fixed-wireless-access",
    "free-adsl"]
    WifiHomeProbes = {"connected":[]}
    
    for probe in pFile["objects"]:
        if probe["status_name"] == "Connected":
            isHome = False
            isWifi = False
            for tag in probe["tags"]:
                for ctag in WifiHomeTags:
                    if tag == ctag:
                        isWifi = True
                        break
                if tag == "home":
                    isHome = True
                if isWifi and isHome:
                    WifiHomeProbes["connected"].append(probe)
                    break

    with open("./wifiHomeProbes.json", "w") as wifiHomeProbeFile:
        json.dump(WifiHomeProbes, wifiHomeProbeFile, indent=4, sort_keys=True)

    print(len(WifiHomeProbes["connected"]))

In [23]:
#function to find all connected home wired probes according to tags given
def findWiredHomeProbes(pFile):
    WiredHomeTags = ["cable","ethernet","wiredethernet","fiber","dsl","ftth","ftth-2","ftth-3","200100-ftth-german-telekom",]
    WiredHomeProbes = {"connected":[]}
    
    for probe in probeFile["objects"]:
        if probe["status_name"] == "Connected":
            isHome = False
            isWifi = False
            for tag in probe["tags"]:
                for ctag in WiredHomeTags:
                    if tag == ctag:
                        isWifi = True
                        break
                if tag == "home":
                    isHome = True
                if isWifi and isHome:
                    WiredHomeProbes["connected"].append(probe)
                    break

    with open("./wiredHomeProbes.json", "w") as wiredHomeProbeFile:
        json.dump(WiredHomeProbes, wiredHomeProbeFile, indent=4, sort_keys=True)

    print(len(WiredHomeProbes["connected"]))

In [24]:
# function to filter probes to exclude probes that contain wrong tags
def filterFiles(addTags,fileName,newFileName):
    rejectedTags = ["datacenter","datacentre","datacenter-2","datacenter-3","mobile-datacentre","gcp","aws","aws-2"]
    if addTags != []:
        rejectedTags = rejectedTags + addTags
    confirmedProbes = {"confirmedProbes":[]}

    with open(fileName, 'r') as f:
        probeFile = json.load(f)

    for probe in probeFile["connected"]:
        reject = False
        for tag in probe["tags"]:
            for ctag in rejectedTags:
                if tag == ctag:
                    reject = True
                    break
            if reject:
                break
        if reject == False:
            confirmedProbes["confirmedProbes"].append(probe)

    with open(newFileName, "w") as confirmedProbesFile:
        json.dump(confirmedProbes, confirmedProbesFile, indent=4, sort_keys=True)

    print(newFileName,":",len(confirmedProbes["confirmedProbes"]))

In [29]:
#Function to sort probes by total country numbers and decreasing the total number of probes if # is higher than 100
def findCountryNo(filename,newFileName):
    with open(filename, 'r') as f:
        probeFile = json.load(f)

    probeByCountry = {}

    for probe in probeFile["confirmedProbes"]:
        if probe["country_code"] in probeByCountry:
            probeByCountry[probe["country_code"]].append(probe)
        else:
            probeByCountry[probe["country_code"]] = [probe]
    print(filename,":countryNo:",len(probeByCountry))
    sortedProbeByCountry = sorted(probeByCountry.items(), key=lambda x: len(x[1]), reverse=True)  

    confProbes = {"confirmedProbes" : []}
    if len(probeFile["confirmedProbes"]) > 100:
        for i in range(100):
            confProbes["confirmedProbes"].append(sortedProbeByCountry[i%len(sortedProbeByCountry)][1][0])
            del sortedProbeByCountry[i%len(sortedProbeByCountry)][1][0]
            if len(sortedProbeByCountry[i%len(sortedProbeByCountry)][1]) < 1:
                del sortedProbeByCountry[i%len(sortedProbeByCountry)]
    else:
        for level1 in sortedProbeByCountry:
            for level2 in level1[1]:
                confProbes["confirmedProbes"].append(level2)

    print(len(confProbes["confirmedProbes"]),newFileName)
    jsonFileName = newFileName + ".json"

    with open(jsonFileName, "w") as probeCountryFile:
        json.dump(confProbes, probeCountryFile, indent=4)
    print(jsonFileName,len(confProbes))

In [26]:
# function used to find 
def FindCloseLAN(lanProbes, wifiProbes, cellProbes):
    lanList = {"connected" : []}
    found100Lan = False
    for lan in lanProbes:
        foundLan = False
        for wifi in wifiProbes:
            if isCloseEnough(lan,wifi):
                for cell in cellProbes:
                    if isCloseEnough(lan,cell):
                        newPair = {"lan": lan, "wifi": wifi, "cell": cell}
                        lanList["connected"].append(newPair)
                        foundLan = True
                        if len(lanList["connected"]) == 100:
                            print("NOW HUNDRED")
                            found100Lan = True
                    if foundLan:
                        break
            if foundLan:
                break
        if found100Lan:
            break

    print(len(lanList["connected"]))
    
    with open("./LanHomeNearby.json", "w") as lanHomeProbeFile:
        json.dump(lanList, lanHomeProbeFile, indent=4, sort_keys=True)

In [27]:
#function to write ids separately to be able to easily use in Atlas Web GUI
def writeIDsCommaSeperated(pFileName):
    with open(pFileName, 'r') as f:
        probeFile = json.load(f)
    print(len(probeFile["confirmedProbes"]))
    IDprint = ""
    for probe in probeFile["confirmedProbes"]:
        IDprint = IDprint + str(probe["id"]) + ", "
    print(IDprint)

In [30]:
with open(probeFileName, 'r') as f:
    probeFile = json.load(f)

findCellularProbes(pFile=probeFile)
findSatelliteProbes(pFile=probeFile)
findWifiHomeProbes(pFile=probeFile)
findWiredHomeProbes(pFile=probeFile)

filterFiles(WifiHomeTags+WiredHomeTags+SatelliteTags,"cellularProbes.json","cellularProbesConf.json")
filterFiles([],"satelliteProbes.json","satelliteProbesConf.json")
filterFiles(WiredHomeTags+CellularTags+SatelliteTags,"wifiHomeProbes.json","wifiHomeProbesConf.json")
filterFiles(WifiHomeTags+CellularTags+SatelliteTags,"wiredHomeProbes.json","wiredHomeProbesConf.json")

findCountryNo("cellularProbesConf.json","cellularProbebyCountry")
findCountryNo("satelliteProbesConf.json","satelliteProbebyCountry")
findCountryNo("wifiHomeProbesConf.json","wifiHomeProbebyCountry")
findCountryNo("wiredHomeProbesConf.json","wiredHomeProbebyCountry")


with open("wiredHomeProbesConf.json", 'r') as f:
        LanProbeFile = json.load(f)

with open("wifiHomeProbebyCountry.json", 'r') as f:
        WifiProbeFile = json.load(f)

with open("cellularProbebyCountry.json", 'r') as f:
        CellProbeFile = json.load(f)

FindCloseLAN(LanProbeFile["confirmedProbes"],WifiProbeFile["confirmedProbes"],CellProbeFile["confirmedProbes"])

writeIDsCommaSeperated("satelliteProbebyCountry.json")
writeIDsCommaSeperated("cellularProbebyCountry.json")
writeIDsCommaSeperated("wifiHomeProbebyCountry.json")
writeIDsCommaSeperated("LanHomeNearby.json")

182
64
684
2014
cellularProbesConf.json : 139
satelliteProbesConf.json : 64
wifiHomeProbesConf.json : 69
wiredHomeProbesConf.json : 1954
cellularProbesConf.json :countryNo: 36
100 cellularProbebyCountry
cellularProbebyCountry.json 1
satelliteProbesConf.json :countryNo: 15
64 satelliteProbebyCountry
satelliteProbebyCountry.json 1
wifiHomeProbesConf.json :countryNo: 26
69 wifiHomeProbebyCountry
wifiHomeProbebyCountry.json 1
wiredHomeProbesConf.json :countryNo: 76
100 wiredHomeProbebyCountry
wiredHomeProbebyCountry.json 1


FileNotFoundError: [Errno 2] No such file or directory: 'CMB-assignment3/wiredHomeProbesConf.json'